In [51]:
from login_details import engine1

In [52]:
with engine1.connect() as con:
    select_statement = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw_1
    ORDER BY datetime ASC
    LIMIT 1000
    """)



In [53]:
# setting value for the first iteration of the variable previous_row
previous_row = -1

# iterate through the database rows
for row in select_statement:

    # previous_row needs a value for first iteration. After this, the value -1 will be overwritten by the actual row value
    if previous_row == -1:
        previous_row = row[1]

    elif row[1] == previous_row:
        with engine1.connect() as con:
            delete_statement = con.execute("""
            DELETE FROM dg_ln_stoppages_raw_1
            WHERE id = (%s);
            """, (row[0]))

KeyboardInterrupt: 